In [2]:
# Unzipping csv.gz and tsv.gz files
!find . -name '*.csv.gz' -exec gzip -d {} \;
!find . -name '*.tsv.gz' -exec gzip -d {} \;

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
